# Explorações iniciais

**Objetivo do projeto**: Utilizar a abordagem de "hierarchical forecasting" para projetar múltiplas séries temporais

## 0. Setup

In [1]:
%load_ext autotime

time: 181 µs (started: 2023-12-28 16:14:29 -03:00)


In [2]:
import pandas as pd
import numpy as np

time: 305 ms (started: 2023-12-28 16:14:29 -03:00)


## 1. Dados: vendas de roupas no varejo

In [3]:
dados = pd.read_csv('https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-hierarchical-forecasting/main/retail-usa-clothing.csv')

dados.head()

,date,state,item,quantity,region,country
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic,USA
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic,USA
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic,USA
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic,USA
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic,USA


time: 968 ms (started: 2023-12-28 16:14:29 -03:00)


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388024 entries, 0 to 388023
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      388024 non-null  object
 1   state     388024 non-null  object
 2   item      388024 non-null  object
 3   quantity  388024 non-null  int64 
 4   region    388024 non-null  object
 5   country   388024 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.8+ MB
time: 51.7 ms (started: 2023-12-28 16:14:30 -03:00)


## 2. Explorações iniciais

### 2.1. Início e fim do dataset

In [5]:
dados['date']\
    .agg(['min', 'max'])

min    1997-11-25
max    2009-07-28
Name: date, dtype: object

time: 40.9 ms (started: 2023-12-28 16:14:30 -03:00)


### 2.2. Valores únicos e frequência por variável

**Highlights**: 

- Temos 4.264 dias distintos. Entretanto, a quantidade de dias entre 25/11/1997 e 28/07/2009 nos aponta 4350. Logo, existem dias sem vendas.
- NewYork e Alabama são os estados que aparecem com mais frequência nos dados. Vermont é o que menos aparece.
- Poucas frequências nas vendas de sapatos masculinos
- SouthCentral é a região/regional que aparece com mais frequência. EastNorthCentral a que menos aparece.

In [6]:
for col in dados.columns:

    print(f'>>>>>>>>>> Coluna: {col}\n')
    print(f'Quantidade de valores únicos: {dados[col].nunique()}')

    print(f'Tabela de frequência:')
    
    display(dados[col].value_counts())

    print('-' * 10, '\n')

>>>>>>>>>> Coluna: date

Quantidade de valores únicos: 4264
Tabela de frequência:


date
1997-11-25    91
2005-09-11    91
2005-08-28    91
2005-08-29    91
2005-08-30    91
              ..
2001-10-22    91
2001-10-23    91
2001-10-24    91
2001-10-25    91
2009-07-28    91
Name: count, Length: 4264, dtype: int64

---------- 

>>>>>>>>>> Coluna: state

Quantidade de valores únicos: 18
Tabela de frequência:


state
NewYork         25584
Alabama         25584
Ohio            21320
Maine           21320
Connecticut     21320
Oregon          21320
RhodeIsland     21320
Tennessee       21320
California      21320
Pennsylvania    21320
NewJersey       21320
Indiana         21320
Hawaii          21320
Illinois        21320
Mississippi     21320
Kentucky        21320
Alaska          21320
Vermont         17056
Name: count, dtype: int64

---------- 

>>>>>>>>>> Coluna: item

Quantidade de valores únicos: 6
Tabela de frequência:


...

---------- 

>>>>>>>>>> Coluna: region

Quantidade de valores únicos: 5
Tabela de frequência:


region
SouthCentral        89544
Pacific             85280
NewEngland          81016
Mid-Alantic         68224
EastNorthCentral    63960
Name: count, dtype: int64

---------- 

>>>>>>>>>> Coluna: country

Quantidade de valores únicos: 1
Tabela de frequência:


country
USA    388024
Name: count, dtype: int64

---------- 

time: 195 ms (started: 2023-12-28 16:14:30 -03:00)


### 2.3. Total de vendas

In [7]:
dados['quantity'].sum()

19208921

time: 3.17 ms (started: 2023-12-28 16:14:31 -03:00)


### 2.4. Total de vendas por variável

**Highlights**: 

- As datas mais recentes são as que possuem mais vendas
- Tennessee (1.351.608), Kentucky (1.298.425) e California (1.173.635) são os estados com mais vendas
- Vestimentas de mulheres são os itens mais vendidos (11.283.595)
- SouthCentral é a região/regional com maior quantidade de vendas (4.792.847)

In [8]:
for col in dados.columns:

    print(f'>>>>>>>>>> Coluna: {col}\n')

    df_temp = dados\
        .groupby(col, dropna = False, as_index = False)['quantity']\
        .sum()\
        .sort_values(by = 'quantity', ascending = False)

    display(df_temp)

>>>>>>>>>> Coluna: date



,date,quantity
4261,2009-07-26,6555
4241,2009-07-06,6551
4252,2009-07-17,6550
4249,2009-07-14,6538
4260,2009-07-25,6536
...,...,...
3,1997-11-28,3123
4,1997-11-29,3119
0,1997-11-25,3115
2,1997-11-27,3101


>>>>>>>>>> Coluna: state



,state,quantity
16,Tennessee,1351608
7,Kentucky,1298425
2,California,1173635
14,Pennsylvania,1166056
15,RhodeIsland,1151458
1,Alaska,1149948
17,Vermont,1109734
11,NewYork,1108082
9,Mississippi,1100123
0,Alabama,1042691


>>>>>>>>>> Coluna: item



...

>>>>>>>>>> Coluna: region



,region,quantity
4,SouthCentral,4792847
3,Pacific,4188458
2,NewEngland,4058463
1,Mid-Alantic,3304925
0,EastNorthCentral,2864228


>>>>>>>>>> Coluna: country



,country,quantity
0,USA,19208921


time: 201 ms (started: 2023-12-28 16:14:31 -03:00)


### 2.5. Temos nulos?

No

In [9]:
dados.isnull().sum()

date        0
state       0
item        0
quantity    0
region      0
country     0
dtype: int64

time: 52.7 ms (started: 2023-12-28 16:14:31 -03:00)


### X. TO-DOs:

- Corrigir o tipo da variável data
- Excluir a coluna country
- 